In [1]:
!pip install groq python-dotenv numpy tqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [groq]2m 7/12 [pydantic]


In [3]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq(api_key=os.getenv("GROQ_API_KEY"))
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

/Users/namkuhyuk-uimacbook/Desktop/YBIGTA_newbie_assignment/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama3-8b-8192"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [5]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello! I'm excited to help you with any math problems you have. What kind of math are you working on? Do you have a specific problem you're stuck on or a topic you'd like to practice? Let me know and I'll do my best to assist you!


#### GSM8K 데이터셋 확인해보기

In [6]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [7]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [8]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama3-8b-8192",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer = extract_final_answer(response)

            if isinstance(predicted_answer, str):
                predicted_answer = float(predicted_answer.replace(",", ""))
            
            diff = abs(predicted_answer - correct_answer)
            is_correct = diff < 1e-5 if predicted_answer is not None else False
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [9]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [9]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [10]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

 50%|█████     | 5/10 [00:03<00:02,  1.81it/s]

Progress: [5/10]
Current Acc.: [20.00%]


100%|██████████| 10/10 [00:06<00:00,  1.61it/s]

Progress: [10/10]
Current Acc.: [30.00%]


In [ ]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!


In [11]:
# 0 shot
PROMPT = construct_direct_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_0.txt")

 10%|█         | 5/50 [00:04<00:33,  1.36it/s]

Progress: [5/50]
Current Acc.: [40.00%]


 20%|██        | 10/50 [00:06<00:19,  2.01it/s]

Progress: [10/50]
Current Acc.: [30.00%]


 30%|███       | 15/50 [00:09<00:20,  1.69it/s]

...

Progress: [40/50]
Current Acc.: [15.00%]


 90%|█████████ | 45/50 [01:01<00:13,  2.62s/it]

Progress: [45/50]
Current Acc.: [15.56%]


100%|██████████| 50/50 [01:13<00:00,  1.47s/it]

Progress: [50/50]
Current Acc.: [18.00%]


In [12]:
# 3 shot
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_3.txt")

 10%|█         | 5/50 [00:04<00:31,  1.42it/s]

Progress: [5/50]
Current Acc.: [20.00%]


 20%|██        | 10/50 [00:11<01:10,  1.75s/it]

Progress: [10/50]
Current Acc.: [30.00%]


 30%|███       | 15/50 [00:24<01:28,  2.53s/it]

...

Progress: [40/50]
Current Acc.: [20.00%]


 90%|█████████ | 45/50 [01:48<00:14,  2.83s/it]

Progress: [45/50]
Current Acc.: [20.00%]


100%|██████████| 50/50 [02:02<00:00,  2.45s/it]

Progress: [50/50]
Current Acc.: [22.00%]


In [13]:
# 5 shot
PROMPT = construct_direct_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_5.txt")

 10%|█         | 5/50 [00:08<01:22,  1.83s/it]

Progress: [5/50]
Current Acc.: [40.00%]


 20%|██        | 10/50 [00:17<01:20,  2.00s/it]

Progress: [10/50]
Current Acc.: [20.00%]


 30%|███       | 15/50 [00:31<01:30,  2.58s/it]

...

Progress: [40/50]
Current Acc.: [22.50%]


 90%|█████████ | 45/50 [01:56<00:17,  3.46s/it]

Progress: [45/50]
Current Acc.: [20.00%]


100%|██████████| 50/50 [02:15<00:00,  2.70s/it]

Progress: [50/50]
Current Acc.: [20.00%]


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [10]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = "Instruction:\nSolve the following mathematical questions. Think step by step and provide a final answer after the tag 'Answer:'.\n"
    #TODO: 프롬프트를 작성해주세요!

    for i in range(num_examples):
        idx = sampled_indices[i]
        cur_question = train_dataset['question'][idx]
        cur_answer = train_dataset['answer'][idx]

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer: {cur_answer}\n"

        #TODO: CoT example을 만들어주세요!

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [ ]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

In [21]:
# 0 shot
PROMPT = construct_CoT_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_0.txt")

 10%|█         | 5/50 [00:04<00:35,  1.25it/s]

Progress: [5/50]
Current Acc.: [60.00%]


 20%|██        | 10/50 [00:08<00:37,  1.06it/s]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [00:16<00:51,  1.46s/it]

...

Progress: [40/50]
Current Acc.: [75.00%]


 90%|█████████ | 45/50 [01:30<00:16,  3.32s/it]

Progress: [45/50]
Current Acc.: [73.33%]


100%|██████████| 50/50 [01:45<00:00,  2.10s/it]

Progress: [50/50]
Current Acc.: [74.00%]


In [13]:
# 3 shot
PROMPT = construct_CoT_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_3.txt")

 10%|█         | 5/50 [00:05<00:38,  1.17it/s]

Progress: [5/50]
Current Acc.: [60.00%]


 20%|██        | 10/50 [00:52<06:20,  9.52s/it]

Progress: [10/50]
Current Acc.: [50.00%]


 30%|███       | 15/50 [01:51<06:35, 11.30s/it]

...

Progress: [40/50]
Current Acc.: [57.50%]


 90%|█████████ | 45/50 [07:20<00:55, 11.09s/it]

Progress: [45/50]
Current Acc.: [57.78%]


100%|██████████| 50/50 [08:16<00:00,  9.94s/it]

Progress: [50/50]
Current Acc.: [60.00%]


In [14]:
# 5 shot
PROMPT = construct_CoT_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_5.txt")

 10%|█         | 5/50 [00:54<08:33, 11.42s/it]

Progress: [5/50]
Current Acc.: [60.00%]


 20%|██        | 10/50 [01:59<08:32, 12.82s/it]

Progress: [10/50]
Current Acc.: [60.00%]


 30%|███       | 15/50 [03:04<07:41, 13.20s/it]

...

Progress: [40/50]
Current Acc.: [62.50%]


 90%|█████████ | 45/50 [09:12<01:01, 12.25s/it]

Progress: [45/50]
Current Acc.: [66.67%]


100%|██████████| 50/50 [10:15<00:00, 12.30s/it]

Progress: [50/50]
Current Acc.: [66.00%]


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [28]:
### 자유롭게 수정해도 됩니다! 완전히 새로 함수를 만들어도 돼요.
def construct_my_prompt(example_list: List[str], num_examples: int = 3):
    train_dataset = gsm8k_train
    # TODO: 구현해주세요!

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\n For each problem, first solve it step-by-step using logical reasoning.\n Then review your own answer to make sure it makes sense. If needed, revise it.\nFinally, write the correct result in the format: 'Final Answer: ####'.\n"
    

    for i in range(num_examples):
        idx = sampled_indices[i]
        cur_question = train_dataset['question'][idx]
        cur_answer = train_dataset['answer'][idx]
        numeric_answer = cur_answer.split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer: {cur_answer}\n"
        prompt += f"{cur_answer}\n"
        prompt += f"Review: The reasoning seems consistent.\n"
        prompt += f"Final Answer: #### {numeric_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [ ]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!

In [29]:
# 0 shot
PROMPT = construct_my_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "My_prompting_0.txt")

 10%|█         | 5/50 [00:04<00:38,  1.17it/s]

Progress: [5/50]
Current Acc.: [60.00%]


 20%|██        | 10/50 [01:00<07:02, 10.57s/it]

Progress: [10/50]
Current Acc.: [60.00%]


 30%|███       | 15/50 [02:03<07:13, 12.40s/it]

...

Progress: [40/50]
Current Acc.: [60.00%]


 90%|█████████ | 45/50 [08:05<01:01, 12.35s/it]

Progress: [45/50]
Current Acc.: [62.22%]


100%|██████████| 50/50 [09:06<00:00, 10.93s/it]

Progress: [50/50]
Current Acc.: [64.00%]


In [30]:
# 3 shot
PROMPT = construct_my_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "My_prompting_3.txt")

 10%|█         | 5/50 [00:04<00:46,  1.03s/it]

Progress: [5/50]
Current Acc.: [60.00%]


 20%|██        | 10/50 [00:41<05:13,  7.83s/it]

Progress: [10/50]
Current Acc.: [60.00%]


 30%|███       | 15/50 [01:33<05:58, 10.24s/it]

...

Progress: [40/50]
Current Acc.: [67.50%]


 90%|█████████ | 45/50 [06:36<00:50, 10.15s/it]

Progress: [45/50]
Current Acc.: [68.89%]


100%|██████████| 50/50 [07:27<00:00,  8.96s/it]

Progress: [50/50]
Current Acc.: [70.00%]


In [31]:
# 5 shot
PROMPT = construct_my_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "My_prompting_5.txt")

 10%|█         | 5/50 [01:02<11:35, 15.46s/it]

Progress: [5/50]
Current Acc.: [60.00%]


 20%|██        | 10/50 [01:43<07:28, 11.22s/it]

Progress: [10/50]
Current Acc.: [50.00%]


 30%|███       | 15/50 [02:48<07:34, 12.98s/it]

...

Progress: [40/50]
Current Acc.: [65.00%]


 90%|█████████ | 45/50 [09:11<01:03, 12.80s/it]

Progress: [45/50]
Current Acc.: [66.67%]


100%|██████████| 50/50 [10:14<00:00, 12.30s/it]

Progress: [50/50]
Current Acc.: [68.00%]


### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!